In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
import tensorflow_datasets as tfds


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
train_df = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip")
test_df =  pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip")
result_df = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/sample_submission.csv.zip")
train_df["color"].replace({"clear": (1/6), "white": (2/6),"blood":(3/6),"green":(4/6),"blue":(5/6),"black": 1 }, inplace=True)
test_df["color"].replace({"clear": (1/6), "white": (2/6),"blood":(3/6),"green":(4/6),"blue":(5/6),"black": 1 }, inplace=True)

train_df["type"].replace({"Ghoul": 0, "Goblin": 1,"Ghost":2}, inplace=True)
features = train_df.copy()
ids = features.pop('id')
labels = features.pop('type')

features = np.array(features)

test_features = test_df.copy()
test_ids = test_features.pop('id')
test_features = np.array(test_features)

monsterClasses = ["Ghoul","Goblin","Ghost"]

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation=tf.nn.relu, input_shape=(5,)),#1st layer
    tf.keras.layers.Dense(25, activation=tf.nn.relu), #2nd layer ,just set this line as a comment to have only 1 layer
    tf.keras.layers.Dense(3)
])
predictions = model(features)
tf.nn.softmax(predictions)

In [6]:
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1))
model.fit(features,labels, epochs= 100)

In [7]:
predictions = model(features)
print(predictions)

In [8]:
print("Prediction: {}".format(tf.argmax(predictions, axis=1)))
print(labels)

In [9]:
test_predict = tf.convert_to_tensor(test_features)
predictions = model(test_predict, training=False)
test_types = []
for i, logits in enumerate(predictions):
    class_idx = tf.argmax(logits).numpy()
    p = tf.nn.softmax(logits)[class_idx]
    name = monsterClasses[class_idx]
    test_types.append(name)
    print("Monster {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))

In [10]:
result_df.type = test_types
result_df["type"].replace({ 0 : "Ghoul", 1 : "Goblin" ,2 : "Ghost"}, inplace=True)
print(result_df)

In [11]:
filename = "submission.csv"
#print(filename)
result_df.to_csv(filename,index = False)